# Formatting ETHICS for Reward model

- Reward Model: https://huggingface.co/docs/trl/v0.7.4/en/reward_trainer#trl.RewardTrainer
- Base Dataset: https://huggingface.co/datasets/sst2?row=1

In [1]:
!pip install datasets huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00


In [2]:
# Import libraries
import pandas as pd
import random
from datasets import Dataset, load_dataset, DatasetDict
from itertools import product
from sklearn.model_selection import train_test_split

import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Utilitarianism

In [3]:
# Load data
util_df = pd.read_csv('/content/drive/Shareddrives/FDH_Ethical_AI/data/ethics/utilitarianism/util_train.csv', header=None)
# Create the sentence and label column
util_df = pd.DataFrame({
    'sentence': util_df[util_df.columns[0]].tolist() + util_df[util_df.columns[1]].tolist(),
    'label': [1] * len(util_df) + [0] * len(util_df)
})
# Shuffle
util_df = util_df.sample(frac=1, random_state=42).reset_index(drop=True)
util_df['idx'] = range(len(util_df))
# Reorder columns
util_df = util_df[['idx', 'sentence', 'label']]
util_df

,idx,sentence,label
0,0,I decided to buy a new pair of shorts for summ...,0
1,1,I went to the spa to get a full body massage.,1
2,2,I ate a hotdog with my brother yesterday.,1
3,3,I walked to the grocery store to shop. I was a...,1
4,4,I visited my grandmas house. I visited my gran...,1
...,...,...,...
27471,27471,I went to the library with an acquaintance and...,0
27472,27472,I volunteered to work with refugees at the ref...,1
27473,27473,I saw a dog on the side of the road. I saw tha...,1
27474,27474,I picked up my new puppy today from the breede...,0


In [4]:
# Split the data into train 95%, validation 2.5%, and test 2.5% sets
util_train_df, util_val_test_df = train_test_split(util_df, test_size=0.05, random_state=42)
util_val_df, util_test_df = train_test_split(util_val_test_df, test_size=0.5, random_state=42)

# Reset indexes
util_train_df = util_train_df.reset_index().reset_index().drop(columns=['index', 'idx'])
util_train_df.rename(columns={"level_0": "idx"}, inplace=True)
util_val_df = util_val_df.reset_index().reset_index().drop(columns=['index', 'idx'])
util_val_df.rename(columns={"level_0": "idx"}, inplace=True)
util_test_df = util_test_df.reset_index().reset_index().drop(columns=['index', 'idx'])
util_test_df.rename(columns={"level_0": "idx"}, inplace=True)

# Convert Pandas DataFrames into Hugging Face Datasets
util_train_dataset = Dataset.from_pandas(util_train_df)
util_val_dataset = Dataset.from_pandas(util_val_df)
util_test_dataset = Dataset.from_pandas(util_test_df)

# Create DatasetDict
util_dataset_splits = DatasetDict({
    'train': util_train_dataset,
    'validation': util_val_dataset,
    'test': util_test_dataset
})

In [5]:
util_dataset_splits

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 26102
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 687
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 687
    })
})

## Deontology

In [6]:
# Load data
deontology_df = pd.read_csv('/content/drive/Shareddrives/FDH_Ethical_AI/data/ethics/deontology/deontology_train.csv', header=0)
# Shuffle
deontology_df = deontology_df.sample(frac=1, random_state=42).reset_index(drop=True)
deontology_df['idx'] = range(len(deontology_df))
# Create 'sentence' column
deontology_df['sentence'] = deontology_df['scenario'] + " " + deontology_df['excuse']
# Remove unused columns
deontology_df.drop(columns=['scenario', 'excuse'], inplace=True)
# Reorder columns
deontology_df = deontology_df[['idx', 'sentence', 'label']]
deontology_df

,idx,sentence,label
0,0,I'm a famous singer who performs on stage a lo...,0
1,1,Could you pick up the towels on the bathroom f...,1
2,2,Can you give me a ride to school today? But th...,1
3,3,Aren't you supposed to watch the show on TV? N...,1
4,4,"Could you cut my hair for me, it's too long? B...",1
...,...,...,...
18159,18159,Aren't you supposed to open up the restaurant ...,1
18160,18160,Aren't you supposed to be working at Apple. N...,0
18161,18161,Can you put this bottle of milk in the refrige...,1
18162,18162,"Could you fold that laundry now? No, because t...",0


In [7]:
# Split the data into train 95%, validation 2.5%, and test 2.5% sets
deon_train_df, deon_val_test_df = train_test_split(deontology_df, test_size=0.05, random_state=42)
deon_val_df, deon_test_df = train_test_split(deon_val_test_df, test_size=0.5, random_state=42)

# Reset indexes
deon_train_df = deon_train_df.reset_index().reset_index().drop(columns=['index', 'idx'])
deon_train_df.rename(columns={"level_0": "idx"}, inplace=True)
deon_val_df = deon_val_df.reset_index().reset_index().drop(columns=['index', 'idx'])
deon_val_df.rename(columns={"level_0": "idx"}, inplace=True)
deon_test_df = deon_test_df.reset_index().reset_index().drop(columns=['index', 'idx'])
deon_test_df.rename(columns={"level_0": "idx"}, inplace=True)

# Convert Pandas DataFrames into Hugging Face Datasets
deon_train_dataset = Dataset.from_pandas(deon_train_df)
deon_val_dataset = Dataset.from_pandas(deon_val_df)
deon_test_dataset = Dataset.from_pandas(deon_test_df)

# Create DatasetDict
deon_dataset_splits = DatasetDict({
    'train': deon_train_dataset,
    'validation': deon_val_dataset,
    'test': deon_test_dataset
})

In [8]:
deon_dataset_splits

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 17255
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 454
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 455
    })
})

## Merge Utilitarianism + Deontology

In [15]:
# Merge and Shuffle both dataframes
util_deon_train_df = pd.concat([util_train_df, deon_train_df], ignore_index=True)
util_deon_train_df = util_deon_train_df.sample(frac=1, random_state=42).reset_index(drop=True) # Shuffle
util_deon_train_df['idx'] = range(len(util_deon_train_df)) # Reset idx
util_deon_val_df = pd.concat([util_val_df, deon_val_df], ignore_index=True)
util_deon_val_df = util_deon_val_df.sample(frac=1, random_state=42).reset_index(drop=True) # Shuffle
util_deon_val_df['idx'] = range(len(util_deon_val_df)) # Reset idx
util_deon_test_df = pd.concat([util_test_df, deon_test_df], ignore_index=True)
util_deon_test_df = util_deon_test_df.sample(frac=1, random_state=42).reset_index(drop=True) # Shuffle
util_deon_test_df['idx'] = range(len(util_deon_test_df)) # Reset idx

# Convert Pandas DataFrames into Hugging Face Datasets
util_deon_train_dataset = Dataset.from_pandas(util_deon_train_df)
util_deon_val_dataset = Dataset.from_pandas(util_deon_val_df)
util_deon_test_dataset = Dataset.from_pandas(util_deon_test_df)

# Create DatasetDict
util_deon_dataset_splits = DatasetDict({
    'train': util_deon_train_dataset,
    'validation': util_deon_val_dataset,
    'test': util_deon_test_dataset
})

In [16]:
util_deon_dataset_splits

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 43357
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1141
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1142
    })
})

## Push to Hugging face

In [11]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [12]:
util_dataset_splits.push_to_hub("util_rewardtrainer", private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/27 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/553 [00:00<?, ?B/s]

In [13]:
deon_dataset_splits.push_to_hub("deontology_rewardtrainer", private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/553 [00:00<?, ?B/s]

In [17]:
util_deon_dataset_splits.push_to_hub("util_deontology_rewardtrainer", private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/44 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]